In [ ]:
import polars as pl

# Load data
df = pl.read_csv("data/health_survey.csv")
reverse_df = pl.read_csv("data/ReverseCodingItems.csv")

# Define reverse coding mapping
reverse_map = {
    "Strongly Disagree": 5,
    "Somewhat Disagree": 4,
    "Neither Agree nor Disagree": 3,
    "Somewhat Agree": 2,
    "Strongly Agree": 1
}

# Apply reverse coding where necessary
df = df.with_columns([
    pl.when(reverse_df["Needs Reverse Coding"] == "Yes")
    .then(pl.col("Response").map_dict(reverse_map))
    .otherwise(pl.col("Response").cast(int))
    .alias("Recoded Value")
])

# Create Temp Coded and Reverse Coded Value columns
df = df.with_columns([
    pl.col("Response").map_dict({
        "Strongly Disagree": 1,
        "Somewhat Disagree": 2,
        "Neither Agree nor Disagree": 3,
        "Somewhat Agree": 4,
        "Strongly Agree": 5
    }).alias("Temp Coded Value"),
    pl.when(pl.col("Needs Reverse") == "Yes")
    .then(pl.col("Response").map_dict(reverse_map))
    .otherwise(pl.col("Temp Coded Value"))
    .alias("Final Coded Value")
])

# Recategorize question types
df = df.with_columns([
    pl.col("Question Type").map_dict({
        "Type1": "F1",
        "Type2": "F2",
        "Type3": "F3",
        # Add remaining mappings here
    }).alias("Recoded Question Type")
])

# Write to CSV
df.write_csv("data/health_survey_summary.csv")